<a href="https://colab.research.google.com/github/jahnavimidde/VsemML/blob/main/DatasetsML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
df=pd.read_csv("titanic_train.csv")
print(df)
df['Age'].fillna(df['Age'].median(), inplace=True)  # Median is safer for skewed data
df['Fare'].fillna(df['Fare'].mean(), inplace=True)  # Mean
df.ffill()
print(df.isnull().sum)



     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

/tmp/ipython-input-2684091815.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)  # Median is safer for skewed data
/tmp/ipython-input-2684091815.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col]

In [5]:
#CONSTANT FEATURES
from sklearn.model_selection import train_test_split
[col for col in df.columns if df[col].isnull().sum()>0]
X=df.drop('Survived',axis=1)
X=pd.get_dummies(X,drop_first=True)
y=df['Survived']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
constant_features=[features for features in X_train.columns if X_train[features].std()==0]
print(len(constant_features))
X_train.drop(columns=constant_features, inplace=True)
X_test.drop(columns=constant_features, inplace=True)

311


In [6]:
#quasi-constant features
X = df.drop('Survived', axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
quasi_constant_features=[]
for feature in X_train.columns:
  predominant=(X_train[feature].value_counts()/float(len(X_train))).sort_values(ascending=False).values[0]
  if(predominant>0.99):
    quasi_constant_features.append(feature)
print(len(quasi_constant_features))
print(quasi_constant_features)
X_train.drop(columns=quasi_constant_features, inplace=True)
X_test.drop(columns=quasi_constant_features, inplace=True)

0
[]


In [7]:
#duplicated features
dup=[]
for i in range(0,len(X_train.columns)):
  col1=X_train.columns[i]
  for col2 in X_train.columns[i+1:]:
    if(X_train[col1].equals(X_train[col2])):
      dup.append(col2)
print(len(dup))
print(dup)
X_train.drop(columns=dup, inplace=True)
X_test.drop(columns=dup, inplace=True)

0
[]


In [8]:
# Correlation
grouped_feature_ls = []
correlated_groups = []

def correlation(dataset, threshold):
    col_corr = set()
    numeric_df = dataset.select_dtypes(include=['number'])  # Only numeric columns
    col_matrix = numeric_df.corr()
    for i in range(len(col_matrix.columns)):
        for j in range(i):
            if abs(col_matrix.iloc[i, j]
                   ) > threshold:
                colname = col_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

corr_features = correlation(df, 0.9)
corr_final = set(corr_features)
print(len(corr_final))
X_train.drop(columns=corr_final, inplace=True)
X_test.drop(columns=corr_final, inplace=True)

0


In [9]:
#Mutual Information
from sklearn.feature_selection import mutual_info_classif, SelectKBest
import pandas as pd

X = df.drop('Survived', axis=1)
X = pd.get_dummies(X, drop_first=True)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

mi_scores = mutual_info_classif(X.fillna(0), y)
mi_scores = pd.Series(mi_scores, index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
print("Mutual Information Scores:\n", mi_scores)

X_train_encoded = pd.get_dummies(X_train, drop_first=True).fillna(0)
sel_ = SelectKBest(mutual_info_classif, k=10).fit(X_train_encoded, y_train)

top_features = X_train_encoded.columns[sel_.get_support()]
print("\nTop 10 Features:\n", top_features)

Mutual Information Scores:
 Fare                                 0.142929
Sex_male                             0.141331
Pclass                               0.062980
Name_Pinsky, Mrs. (Rosa)             0.051293
Ticket_347076                        0.050191
                                       ...   
Ticket_19943                         0.000000
Name_Leader, Dr. Alice (Farnham)     0.000000
Name_Dahlberg, Miss. Gerda Ulrika    0.000000
Name_Daly, Mr. Peter Denis           0.000000
Ticket_12749                         0.000000
Length: 1725, dtype: float64

Top 10 Features:
 Index(['Pclass', 'Fare', 'Name_Harper, Rev. John',
       'Name_Madsen, Mr. Fridtjof Arne', 'Name_Moss, Mr. Albert Johan',
       'Name_Yasbeck, Mrs. Antoni (Selini Alexander)', 'Sex_male',
       'Ticket_349217', 'Ticket_A/5 3902', 'Cabin_C50'],
      dtype='object')


In [10]:
#CHI SQUARE TEst
from scipy.stats import chi2_contingency
result=[]
for feature in X_train.columns:
  crosstab=pd.crosstab(X_train[feature],df['Survived'])
  chi2_stat,p_val,dof,expected=chi2_contingency(crosstab)
  if p_val<0.05:
    result.append(feature)
print(result)
X_train.drop(columns=feature, inplace=True)
X_test.drop(columns=feature, inplace=True)

['Pclass', 'SibSp', 'Parch', 'Fare', 'Sex_male', 'Embarked_S']


In [11]:
#Anova
from sklearn.feature_selection import f_classif, SelectKBest
anova_selector = SelectKBest(score_func=f_classif, k=10)
anova_selector.fit(X_train, y_train)

top_features = X_train.columns[anova_selector.get_support()]
print(top_features)

anova_scores = pd.Series(anova_selector.scores_, index=X_train.columns)
anova_scores = anova_scores.sort_values(ascending=False)
print(anova_scores.head(10))

Index(['Pclass', 'Fare', 'Sex_male', 'Ticket_113760', 'Ticket_24160',
       'Ticket_2666', 'Ticket_29106', 'Ticket_CA. 2343', 'Cabin_B96 B98',
       'Cabin_E101'],
      dtype='object')
Sex_male           260.604149
Pclass              78.681039
Fare                34.103369
Ticket_113760        4.767074
Ticket_29106         4.767074
Cabin_E101           4.767074
Ticket_24160         4.767074
Cabin_B96 B98        4.767074
Ticket_2666          4.767074
Ticket_CA. 2343      4.514871
dtype: float64


/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [   7    8   12   17   18   30   32   37   43   47   50   54   55   57
   58   66   67   69   73   76   80   87   90   91   94  102  104  109
  112  114  115  120  124  130  135  138  139  149  152  163  165  168
  171  175  183  185  187  195  196  197  198  207  211  213  216  225
  226  228  237  238  242  247  253  254  257  265  272  273  274  278
  281  284  289  292  305  306  308  311  313  314  319  320  323  324
  327  331  336  339  341  342  344  345  346  349  350  352  353  354
  357  358  361  362  365  369  375  376  388  403  408  409  410  413
  414  417  422  423  428  437  439  440  442  443  449  452  453  454
  455  456  465  467  473  475  476  477  492  495  496  499  509  512
  513  518  523  525  527  529  532  535  538  539  540  545  546  547
  548  551  555  558  559  560  565  567  568  573  576  580  584  587
  591  592  594  602  613  617

In [12]:
import pandas as pd
df1=pd.read_csv("Santander Customer Satisfaction_train.csv")
print(df1)

         ID  var3  var15  imp_ent_var16_ult1  imp_op_var39_comer_ult1  \
0         1     2     23                 0.0                      0.0   
1         3     2     34                 0.0                      0.0   
2         4     2     23                 0.0                      0.0   
3         8     2     37                 0.0                    195.0   
4        10     2     39                 0.0                      0.0   
...     ...   ...    ...                 ...                      ...   
9397  18886     2     23                 0.0                      0.0   
9398  18891     2     35                 0.0                      0.0   
9399  18896     2     48                 0.0                      0.0   
9400  18898     2     65                 0.0                      0.0   
9401  18901     2     23                 0.0                      0.0   

      imp_op_var39_comer_ult3  imp_op_var40_comer_ult1  \
0                         0.0                      0.0   
1      

In [13]:
#CONSTANT FEATURES
from sklearn.model_selection import train_test_split
[col for col in df1.columns if df1[col].isnull().sum()>0]
X=df1.drop('TARGET',axis=1)
X=pd.get_dummies(X,drop_first=True)
y=df1['TARGET']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
constant_features=[features for features in X_train.columns if X_train[features].std()==0]
print(len(constant_features))
X_train.drop(columns=constant_features, inplace=True)
X_test.drop(columns=constant_features, inplace=True)

85


In [14]:
#quasi-constant features
X = df1.drop('TARGET', axis=1)
y = df1['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
quasi_constant_features=[]
for feature in X_train.columns:
  predominant=(X_train[feature].value_counts()/float(len(X_train))).sort_values(ascending=False).values[0]
  if(predominant>0.99):
    quasi_constant_features.append(feature)
print(len(quasi_constant_features))
print(quasi_constant_features)
X_train.drop(columns=quasi_constant_features, inplace=True)
X_test.drop(columns=quasi_constant_features, inplace=True)

218
['imp_op_var40_comer_ult1', 'imp_op_var40_comer_ult3', 'imp_op_var40_efect_ult1', 'imp_op_var40_efect_ult3', 'imp_op_var40_ult1', 'imp_sal_var16_ult1', 'ind_var1', 'ind_var2_0', 'ind_var2', 'ind_var6_0', 'ind_var6', 'ind_var13_medio_0', 'ind_var13_medio', 'ind_var14', 'ind_var17_0', 'ind_var17', 'ind_var18_0', 'ind_var18', 'ind_var19', 'ind_var20_0', 'ind_var20', 'ind_var27_0', 'ind_var28_0', 'ind_var28', 'ind_var27', 'ind_var29_0', 'ind_var29', 'ind_var30_0', 'ind_var31_0', 'ind_var31', 'ind_var32_cte', 'ind_var32_0', 'ind_var32', 'ind_var33_0', 'ind_var33', 'ind_var34_0', 'ind_var34', 'ind_var40', 'ind_var41', 'ind_var39', 'ind_var44_0', 'ind_var44', 'ind_var46_0', 'ind_var46', 'num_var1', 'num_var6_0', 'num_var6', 'num_var13_medio_0', 'num_var13_medio', 'num_var14', 'num_var17_0', 'num_var17', 'num_var18_0', 'num_var18', 'num_var20_0', 'num_var20', 'num_op_var40_hace2', 'num_op_var40_hace3', 'num_op_var40_ult1', 'num_op_var40_ult3', 'num_var27_0', 'num_var28_0', 'num_var28', 'nu

In [15]:
#duplicated features
dup=[]
for i in range(0,len(X_train.columns)):
  col1=X_train.columns[i]
  for col2 in X_train.columns[i+1:]:
    if(X_train[col1].equals(X_train[col2])):
      dup.append(col2)
print(len(dup))
print(dup)
X_train.drop(columns=dup, inplace=True)
X_test.drop(columns=dup, inplace=True)

8
['ind_var26', 'ind_var25', 'ind_var37', 'num_var40_0', 'num_var26', 'num_var25', 'num_var37', 'delta_num_aport_var13_1y3']


In [16]:
# Correlation
grouped_feature_ls = []
correlated_groups = []

def correlation(dataset, threshold):
    col_corr = set()
    numeric_df = dataset.select_dtypes(include=['number'])
    col_matrix = numeric_df.corr()
    for i in range(len(col_matrix.columns)):
        for j in range(i):
            if abs(col_matrix.iloc[i, j]) > threshold:
                colname = col_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

corr_features = correlation(df1, 0.9)
corr_final = set(corr_features)
print(len(corr_final))

X_train.drop(columns=corr_final, inplace=True, errors='ignore')
X_test.drop(columns=corr_final, inplace=True, errors='ignore')

155


In [17]:
#Mutual Information
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.model_selection import train_test_split
import pandas as pd

df1 = df1.dropna(subset=['TARGET'])
X = df1.drop('TARGET', axis=1)
X = pd.get_dummies(X, drop_first=True)
y = df1['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

mi_scores = mutual_info_classif(X.fillna(0), y)
mi_scores = pd.Series(mi_scores, index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
print("Mutual Information Scores:\n", mi_scores)

X_train_encoded = pd.get_dummies(X_train, drop_first=True).fillna(0)
sel_ = SelectKBest(mutual_info_classif, k=10).fit(X_train_encoded, y_train)

top_features = X_train_encoded.columns[sel_.get_support()]
print("\nTop 10 Features:\n", top_features)


Mutual Information Scores:
 var15                           0.014898
saldo_var30                     0.010473
num_meses_var5_ult3             0.009795
ind_var30                       0.009510
saldo_medio_var5_ult3           0.009383
                                  ...   
saldo_medio_var8_ult3           0.000000
ind_var1                        0.000000
saldo_medio_var13_medio_ult1    0.000000
imp_op_var40_comer_ult1         0.000000
imp_op_var39_comer_ult3         0.000000
Length: 370, dtype: float64

Top 10 Features:
 Index(['var15', 'ind_var30', 'num_var5', 'num_var30', 'num_var42',
       'saldo_var30', 'num_meses_var5_ult3', 'saldo_medio_var5_hace3',
       'saldo_medio_var5_ult1', 'saldo_medio_var5_ult3'],
      dtype='object')


In [18]:
#Anova
from sklearn.feature_selection import f_classif, SelectKBest
anova_selector = SelectKBest(score_func=f_classif, k=10)
anova_selector.fit(X_train, y_train)

top_features = X_train.columns[anova_selector.get_support()]
print(top_features)

anova_scores = pd.Series(anova_selector.scores_, index=X_train.columns)
anova_scores = anova_scores.sort_values(ascending=False)
print(anova_scores.head(10))

Index(['var15', 'ind_var5', 'ind_var8_0', 'ind_var30', 'num_var5',
       'num_var8_0', 'num_var30', 'num_var42', 'var36', 'num_meses_var5_ult3'],
      dtype='object')
num_meses_var5_ult3    96.932039
ind_var30              88.168061
num_var30              87.714275
ind_var5               80.326183
num_var5               77.908219
num_var42              71.470667
var15                  67.722201
var36                  39.709405
num_var8_0             29.381055
ind_var8_0             29.381055
dtype: float64


/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [ 22  23  37  38  44  45  57  58  59  60  72  73  81  85  86 103 104 110
 111 132 133 134 135 146 147 156 162 163 171 175 180 181 187 190 193 195
 196 202 203 205 206 207 214 215 217 218 219 221 222 223 224 231 235 237
 238 239 240 243 245 246 247 248 249 250 251 253 262 263 270 286 304 306
 307 308 309 314 316 317 318 319 320 321 322 328 349 350 351 352 358] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [19]:
import pandas as pd
df3=pd.read_csv("house_price_train.csv")
print(df3)

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave   NaN      Reg   
1        2          20       RL         80.0     9600   Pave   NaN      Reg   
2        3          60       RL         68.0    11250   Pave   NaN      IR1   
3        4          70       RL         60.0     9550   Pave   NaN      IR1   
4        5          60       RL         84.0    14260   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1455  1456          60       RL         62.0     7917   Pave   NaN      Reg   
1456  1457          20       RL         85.0    13175   Pave   NaN      Reg   
1457  1458          70       RL         66.0     9042   Pave   NaN      Reg   
1458  1459          20       RL         68.0     9717   Pave   NaN      Reg   
1459  1460          20       RL         75.0     9937   Pave   NaN      Reg   

     LandContour Utilities  ... PoolArea PoolQC  Fe

In [20]:
#CONSTANT FEATURES
from sklearn.model_selection import train_test_split
[col for col in df3.columns if df3[col].isnull().sum()>0]
X=df3.drop('SalePrice',axis=1)
X=pd.get_dummies(X,drop_first=True)
y=df3['SalePrice']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
constant_features=[features for features in X_train.columns if X_train[features].std()==0]
print(len(constant_features))
X_train.drop(columns=constant_features, inplace=True)
X_test.drop(columns=constant_features, inplace=True)

4


In [21]:
#quasi-constant features
X = df3.drop('SalePrice', axis=1)
y = df3['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
quasi_constant_features=[]
for feature in X_train.columns:
  predominant=(X_train[feature].value_counts()/float(len(X_train))).sort_values(ascending=False).values[0]
  if(predominant>0.99):
    quasi_constant_features.append(feature)
print(len(quasi_constant_features))
print(quasi_constant_features)
X_train.drop(columns=quasi_constant_features, inplace=True)
X_test.drop(columns=quasi_constant_features, inplace=True)

4
['Street', 'Utilities', 'Condition2', 'PoolArea']


In [22]:
#duplicated features
dup=[]
for i in range(0,len(X_train.columns)):
  col1=X_train.columns[i]
  for col2 in X_train.columns[i+1:]:
    if(X_train[col1].equals(X_train[col2])):
      dup.append(col2)
print(len(dup))
print(dup)
X_train.drop(columns=dup, inplace=True)
X_test.drop(columns=dup, inplace=True)

0
[]


In [23]:
# Correlation
grouped_feature_ls = []
correlated_groups = []

def correlation(dataset, threshold):
    col_corr = set()
    numeric_df = dataset.select_dtypes(include=['number'])
    col_matrix = numeric_df.corr()
    for i in range(len(col_matrix.columns)):
        for j in range(i):
            if abs(col_matrix.iloc[i, j]) > threshold:
                colname = col_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

corr_features = correlation(df3, 0.9)
corr_final = set(corr_features)
print(len(corr_final))

X_train.drop(columns=corr_final, inplace=True, errors='ignore')
X_test.drop(columns=corr_final, inplace=True, errors='ignore')

0


In [24]:
#Mutual Information
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.model_selection import train_test_split
import pandas as pd

df3 = df3.dropna(subset=['SalePrice'])
X = df3.drop('SalePrice', axis=1)
X = pd.get_dummies(X, drop_first=True)
y = df3['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

mi_scores = mutual_info_classif(X.fillna(0), y)
mi_scores = pd.Series(mi_scores, index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
print("Mutual Information Scores:\n", mi_scores)

X_train_encoded = pd.get_dummies(X_train, drop_first=True).fillna(0)
sel_ = SelectKBest(mutual_info_classif, k=10).fit(X_train_encoded, y_train)

top_features = X_train_encoded.columns[sel_.get_support()]
print("\nTop 10 Features:\n", top_features)

Mutual Information Scores:
 Street_Pave              3.334665
Condition2_Norm          3.193806
RoofMatl_CompShg         3.044214
KitchenAbvGr             2.871722
Heating_GasA             2.856910
                           ...   
MiscFeature_TenC         0.000000
MiscFeature_Shed         0.000000
SaleType_Oth             0.000000
SaleCondition_AdjLand    0.000000
SaleCondition_Alloca     0.000000
Length: 245, dtype: float64

Top 10 Features:
 Index(['KitchenAbvGr', 'Street_Pave', 'Condition2_Norm', 'RoofMatl_CompShg',
       'Heating_GasA', 'CentralAir_Y', 'GarageQual_TA', 'GarageCond_TA',
       'PavedDrive_Y', 'SaleType_WD'],
      dtype='object')


In [25]:
#Anova f_regression
from sklearn.feature_selection import f_regression, SelectKBest
import pandas as pd

X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

selector = SelectKBest(score_func=f_regression, k=10)
selector.fit(X_train, y_train)

top_features = X_train.columns[selector.get_support()]
print("\nTop 10 Features (f_regression):\n", top_features)

scores = pd.Series(selector.scores_, index=X_train.columns)
scores = scores.sort_values(ascending=False)
print("\nRegression Feature Scores:\n", scores.head(10))


Top 10 Features (f_regression):
 Index(['OverallQual', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'FullBath',
       'TotRmsAbvGrd', 'GarageCars', 'GarageArea', 'ExterQual_TA',
       'KitchenQual_TA'],
      dtype='object')

Regression Feature Scores:
 OverallQual       1628.570495
GrLivArea         1131.879650
GarageCars         758.024880
GarageArea         727.175552
TotalBsmtSF        654.341155
1stFlrSF           605.576313
ExterQual_TA       499.122655
FullBath           497.954582
TotRmsAbvGrd       431.845019
KitchenQual_TA     376.161106
dtype: float64
